<a href="https://colab.research.google.com/github/gtmray/Covid-19-Nepal/blob/master/covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd
  import numpy as np
except:
  print('IMPORT ERROR!! TYPE !pip install "package-name" to install')

In [0]:
url = 'https://coronanepal.live/' #credits to this site :)
page_content = requests.get(url).content #requesting information from the url
soup = BeautifulSoup(page_content, 'html.parser') #parsing the html content

In [0]:
#function to convert nepali numbers into english
def nep_to_eng(numbers):
  to_return = ''
  for i in numbers:
    conversion = {
        '०':'0',
        '१':'1',
        '२':'2',
        '३':'3',
        '४':'4',
        '५':'5',
        '६':'6',
        '७':'7',
        '८':'8',
        '९':'9',
    }
    to_return = to_return + (conversion.get(i, ''))
  return int(to_return) 


In [4]:
'''Not the good idea to scrap data but works fine for this page'''

test1 = soup.find_all('h1', attrs={'style': 'color:white;'}) #scrapping total tests in nepal
test2 = soup.find_all('h1', attrs={'style': 'color:#f38a03'}) #scrapping infected in nepal and world
test3 = soup.find_all('h1', attrs={'style': 'color:#e60000 !important;'}) #scrapping death in nepal and world
test4 = soup.find_all('h1', attrs={'style': 'color:#70a800 !important;'}) #scrapping recovered in nepal and world

list_of_values = []
for i in test1, test2, test3, test4:
  for j in i:
    j = "".join([k for k in j.string if ',' not in k]) #removing "," from the numbers
    returned = nep_to_eng(j)
    list_of_values.append(returned)

dict_keys = ['Test in Nepal', 'Infected in Nepal', 'Infected in World', 'Death in Nepal',
             'Death in World', 'Recovered in Nepal', 'Recovered in World']
dff = [i for i in zip(dict_keys, list_of_values)] #combining variables and values
df = pd.DataFrame(dff, columns=['Variable', 'Value']) #make dataframe for further manipulation
dfff = pd.pivot_table(df, values='Value', columns='Variable') #rows into columns
pd.set_option('display.max_columns', 20) #to fit while displaying
dfff = dfff[['Test in Nepal', 'Infected in Nepal', 'Death in Nepal', 'Recovered in Nepal', 
             'Infected in World', 'Death in World', 'Recovered in World']] #arranging in some order
print(dfff)

Variable  Test in Nepal  Infected in Nepal  Death in Nepal  \
Value             69587               1572               8   

Variable  Recovered in Nepal  Infected in World  Death in World  \
Value                    220            6263758          373872   

Variable  Recovered in World  
Value                2846713  


In [5]:
t1 = []
t2 = []
t3 = []
t4 = []
t5 = []
t6 = []
select_next = soup.find('table') #Find the table that contents district data
for i in select_next.select('tr'): #For each unique district
  j = i.select('td') #Values of each district
  if len(j)==6:
      t1.append(j[0].find(text=True)) #filter text with text=True
      t2.append(str(j[1].find(text=True))) #district name should be string
      t3.append(int(j[2].find(text=True))) #number of confirmed cases should be numeric
      t4.append(j[3].find(text=True))
      t5.append(j[4].find(text=True))
      t6.append(j[5].find(text=True))
df = pd.DataFrame()
df['District'] = t2
df['Confirmed cases'] = t3
df['Total death'] = t5
df['Total recovered'] = t6
list_eng = []
for i in df['Total recovered']:
  list_eng.append(nep_to_eng(i))
df['Total recovered'] = list_eng
df.sort_values(['Confirmed cases'], ascending=False, ignore_index=True, inplace=True)
print(df)

          District  Confirmed cases Total death  Total recovered
0       kapilbastu              274         ०                  1
1         rautahat              213         ०                  3
2            banke              189         १                 23
3            jhapa              110         ०                  2
4            parsa              107         ०                 93
5          sarlahi               74         ०                  1
6         dhanusha               72         ०                  2
7          saptari               54         ०                  0
8             bara               52         १                  2
9          dailekh               48         १                  0
10       rupandehi               44         ०                 11
11        udayapur               33         ०                 27
12         surkhet               30         ०                  0
13            dang               27         ०                  1
14       mahottari       

In [6]:
#Pie plot
try:
  import chart_studio
except:
  !pip install chart_studio
  import chart_studio
import chart_studio.plotly as py
import plotly.express as px
chart_studio.tools.set_credentials_file(username='raygtm', api_key='Gi1F46rn1rayqGk83fFz4e9')
gbb = df.groupby('District')[['Confirmed cases']].sum().reset_index()
fig = px.pie(gbb, values='Confirmed cases', names='District', title='District wise covid-19 cases')
fig.update_traces(rotation = 310,pull=0.05, textinfo='percent+label')
#fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.update_layout(height=750)
fig.show()
py.iplot(fig, filename='COVID19 District') 

     |████████████████████████████████| 71kB 4.6MB/s 
